# Code generation tests

In [ ]:
import os
import json
from collections import deque
from typing import Dict, List, Optional, Any, Union
import pprint

from langchain_openai import ChatOpenAI
from FPGA_AGI.chains import WebsearchCleaner

from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import chromadb

# import importlib
from FPGA_AGI import tools
from FPGA_AGI import parameters
from FPGA_AGI import utils
from FPGA_AGI import prompts
from FPGA_AGI import chains
from langchain_core.prompts import HumanMessagePromptTemplate
from FPGA_AGI import agents
from FPGA_AGI.agents import Engineer
from FPGA_AGI import utils
from FPGA_AGI.utils import plot_graph

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
#os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

llm = ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0)
bigllm = ChatOpenAI(model='gpt-4-turbo', temperature=0)
gpt4 = ChatOpenAI(model='gpt-4', temperature=0)
gpt4o = ChatOpenAI(model='gpt-4o', temperature=0)

# Improve the h file generation
# Get human objective
# refine it via human in the loop and via adding context (short/long term memory search) -> the output is a json dict of the design
# build the modules and test benches usiiing short and long term memory

### Helper functions:

In [ ]:
webcleaner = WebsearchCleaner.from_llm(llm=gpt4o)

def clean_web(concatenated_content):
    try:
        cleaned_concatenated_content = webcleaner.invoke(concatenated_content)
        cleaned_concatenated_content = cleaned_concatenated_content.cleaned
    except:
        cleaned_concatenated_content = concatenated_content.replace('\n\n', '')
    return cleaned_concatenated_content

def context_from_web(url):
    loader = RecursiveUrlLoader(
        url=url, max_depth=1, extractor=lambda x: Soup(x, "html.parser").text
    )
    docs = loader.load()

    # Add 
    #docs.extend([*docs_pydantic, *docs_sq])

    # Sort the list based on the URLs in 'metadata' -> 'source'
    d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
    d_reversed = list(reversed(d_sorted))

    # Concatenate the 'page_content' of each sorted dictionary
    concatenated_content = "\n\n\n --- \n\n\n".join(
        [doc.page_content for doc in d_reversed]
    )
    cleaned_concatenated_content = clean_web(concatenated_content)
    return cleaned_concatenated_content


# Tests

We have a helper chain that generates requirements from an initial prompt. This helper function may or may not have been mentioned in the paper.
An alternative way to prepare them would be the following:
```python
from FPGA_AGI.chains import Requirements

project_requirements = Requirements()
project_requirements.goals = [list_of_goals]
project_requirements.requirements = [list_of_requirements]
```

## Simple RISC V CPU

In [ ]:
from FPGA_AGI.chains import RequirementChain

requirement_chain = RequirementChain.from_llm(gpt4o)

rescv_concatenated_content = context_from_web("""https://www.fpga4student.com/2017/04/verilog-code-for-16-bit-risc-processor.html""")

riscv_requirements = requirement_chain.invoke(
    {"objective": """Design a simple educational RISC-V CPU. Make sure that your code is synthesizable and can aesily fit on a zynq7 device. The coding language must be hls c++. The code must be supplied with a lot of comments for educational purposes.""", 
     "context" : rescv_concatenated_content}
)

# FFT

In [ ]:
fft_concatenated_content = context_from_web("""https://en.wikipedia.org/wiki/Cooley%E2%80%93Tukey_FFT_algorithm""")

fft_requirements = requirement_chain.invoke(
    {"objective": """Build a 128 point fft circuit which is using two 64 point fft modules (named fft64) to compute the 128 point fft. You do not need to design the 64 point fft devices. You can assume that they just exist.
     the input to the fft module is an array of 128 double precision fixed point real numbers (such as a DSP signal measured elsewhere.) The implementation language must be HLS C++. The design must be optimized for maximum performance (speed.)
     The design must be commented with comments indicating where the design is in fact optimized for performance.""", 
     "context" : fft_concatenated_content})

# Quadruple precision floating point

In [ ]:
float128_concatenated_content = context_from_web("""https://en.wikipedia.org/wiki/Quadruple-precision_floating-point_format""")

float128_requirements = requirement_chain.invoke(
    {"objective": """
     Build a quadrouple precision floating point (not the fixed point) exponentiation module. You have to code this 128 bit floating point representation from scratch, using the context I provided, and code the computations based on this representation.""", 
     "context" : float128_concatenated_content}
)

# Knowledge base (vector database)

In [ ]:
# retriever

embeddings_3_small = OpenAIEmbeddings(model='text-embedding-3-small')
if os.path.isdir('knowledge_base'):
    persistent_client = chromadb.PersistentClient(path="./knowledge_base")
    pdfsearch = Chroma(client=persistent_client, embedding_function=embeddings_3_small)
else:
    pages = []
    for item in ["https://arxiv.org/pdf/1502.07055", "https://arxiv.org/pdf/1810.06885", "https://arxiv.org/pdf/1808.02521", "https://riscv.org/wp-content/uploads/2017/05/riscv-spec-v2.2.pdf"]: # 
        loader = PyPDFLoader(item, extract_images=True) #"https://riscv.org/wp-content/uploads/2017/05/riscv-spec-v2.2.pdf"
        pages += loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    texts = text_splitter.split_documents(pages)
    pdfsearch = Chroma.from_documents(texts, embeddings_3_small, collection_name= "knowledge_base", persist_directory="./knowledge_base")

## All test runs as per the article
Each test consists of:
- requirements: project goals and requirements
- input_context: Project specific informations
- knowledge base (a vector database of all of the necessary information including textbooks and codes)

In [ ]:
#review = None
for requirements, input_context, name in zip(
    [riscv_requirements, fft_requirements, float128_requirements],
    [rescv_concatenated_content, fft_concatenated_content, float128_concatenated_content],
    ['Risc-v', 'FFT', 'Float-128']):
    review = None
    for eval_mod in [bigllm, gpt4o]:
        for model in [llm, bigllm, gpt4o]:
            for i in range(2):
                sol_dir = f'{name}_{model.model_name}_evaluation_{eval_mod.model_name}_{i}'
                R = Engineer(model=model, evaluator_model=eval_mod, retriever=pdfsearch.as_retriever(search_kwargs={"k": 1}), solution_num=sol_dir)
                try:
                    if review:
                        R.lit_review_results = review
                    R.invoke(goals=requirements.goals, requirements=requirements.requirements, input_context= input_context)
                    with open(f"solution_{sol_dir}/goals_requirements.txt", "w") as file:
                        file.write("Goals: \n" + '\n'.join(requirements.goals) + "\nRequirements: \n" + '\n'.join(requirements.requirements))
                    plot_graph(R.hierarchical_solution_result, save_path=f"solution_{sol_dir}/graph.png")
                except:
                    pass
                review = R.lit_review_results